# import packages

In [ ]:
from pandas.io.parsers import read_csv
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import sys, os
mpl.rcParams.update({'figure.max_open_warning': 0})

# load datasets

In [ ]:
pathToData = '/Users/m.blotenburg/Documents/Projects/TCHIC/data/20210215_OUD5651_K27me3K4me3_day5_20201023fixed/'

This is the information from the transcriptome analysis, with V1 and V2 as x- and y- coordinates for the umap.

In [ ]:
cells_clusters = read_csv(pathToData + 'OUD5651_cells_clusters_celltypes_umap.csv', sep = '\t')

In [ ]:
cells_clusters.index = cells_clusters['names']
cells_clusters

In [ ]:
plt.figure(figsize=(8,8))
fig = sns.scatterplot(x=cells_clusters['V1'], y=cells_clusters['V2'], hue = cells_clusters['celltype'], 
                      palette=sns.color_palette("tab20", 13))

Load the chic data too.

In [ ]:
TSS_table_k4 = pd.read_csv(pathToData + 'k4me3/chic/K4me3_TSS_count_table.csv', index_col=(0,1,2,3), 
                        header= 0, low_memory=False)
TSS_table_k4 = TSS_table_k4.iloc[1:]
TSS_table_k4.index.names = ['reference_name','start','end', 'bname']
TSS_table_k4.head()

In [ ]:
TSS_k4 = TSS_table_k4.droplevel(['start', 'end', 'reference_name'], axis=0)

In [ ]:
TSS_k4.T.head()

Merge transcriptome and chic data.

In [ ]:
lower_TSS_K4 = cells_clusters.join(TSS_k4.T,how='inner')

In [ ]:
merged_TSS_K4 = cells_clusters.join(TSS_k4.T,how='outer')
merged_TSS_K4.head()

Do the same for k27 TSS

In [ ]:
TSS_table_k27 = pd.read_csv(pathToData + 'k27me3/chic/K27me3_TSS_count_table.csv', index_col=(0,1,2,3), 
                        header= 0, low_memory=False)
TSS_table_k27 = TSS_table_k27.iloc[1:]
TSS_table_k27.index.names = ['reference_name','start','end', 'bname']
TSS_table_k27.head()

In [ ]:
TSS_k27 = TSS_table_k27.droplevel(['start', 'end', 'reference_name'], axis=0)
lower_TSS_K27 = cells_clusters.join(TSS_k27.T,how='inner')

merged_TSS_K27 = cells_clusters.join(TSS_k27.T,how='outer')
merged_TSS_K27.head()

And k27 gene body

In [ ]:
gb_table_k27 = pd.read_csv(pathToData + 'k27me3/chic/K27me3_genebody_count_table.csv', index_col=(0,1,2,3), 
                        header= 0, low_memory=False)
gb_table_k27 = gb_table_k27.iloc[1:]
gb_table_k27.index.names = ['reference_name','start','end', 'bname']
gb_table_k27.head()

In [ ]:
gb_k27 = gb_table_k27.droplevel(['start', 'end', 'reference_name'], axis=0)
lower_gb_K27 = cells_clusters.join(gb_k27.T,how='inner')

merged_gb_K27 = cells_clusters.join(gb_k27.T,how='outer')
merged_gb_K27.head()

In [ ]:
merged_TSS_K27.columns[4:70]

# plot all the things

To look for genes, use this:

In [ ]:
merged_TSS_K4.filter(regex=r'Hox').columns

This is our color palette

In [ ]:
cols = sns.color_palette('inferno', 9)
cols

Define lists of genes you want to visualize

In [ ]:
markergenes = ['Esrrb', 'Dppa2', 'Dppa4', 'Nanog', 'Dppa5a','Gsc', 'Klf9', 'Klf2', 'Utf1', 'Pim2', 
               'Dppa3', 'Dnmt3b', 'Dazl', #PGC
               'Cdh1', 'Trh', 'Sox17', 'Nodal', 'Epcam', 'Foxa2', 'Lefty1', #endoderm/PGC/node
               'Sox7','Kdr','Car2', #haemogenic
               'Bmp4', 'Gata4', 'Gata6', 'Hand1', 'Hand2', #heart
               'Dll1', 'Hes7', 'Uncx', 'Snai1', 'Hes5', 'Dkk1', #presomitic
               'Eya1', 'Pax3', 'Six1', 'Cer1', #early somite
               'Aldh1a2', 'Notch1', 'Lfng', #wavefront
               'Onecut2', 'Elavl3', 'Crabp1', #neural tube/branchial arches
               'Wnt3a', 'Fgf8', 'T', 'Cdx2', #posterior
               'Sox21', 'Nkx1-2', #neural SOX2 SOX1
               'Prdm1', 'Irx1', 'Eomes', 'Meox1','Tbx4', 'Sox3','H19', 'Xist','Gata2', 'Gata3','Gata4', 'Tet1' #other   
              ]

In [ ]:
hox = ['Hoxb3os', 'Hoxb5os', 'Hoxa5', 'Hoxa2', 'Hoxaas2', 'Hoxaas3', 'Hoxd9',
       'Hoxb5', 'Hoxb4', 'Hoxb3', 'Hoxb2', 'Hoxc6', 'Hoxd3os1', 'Hoxa7',
       'Hoxa1', 'Hoxd3', 'Hoxd4', 'Hoxb9', 'Hoxb8', 'Hoxb7', 'Hoxb6', 'Hoxc8',
       'Hoxa3', 'Hoxa6', 'Hoxa4', 'Hoxd13', 'Hoxd12', 'Hoxd8', 'Hoxd1',
       'Hoxb13', 'Hoxb1', 'Hoxc9', 'Hoxd10', 'Hoxc4', 'Hoxc10', 'Hoxc5',
       'Hoxa13', 'Hoxa10', 'Hoxc13', 'Hoxa9', 'Hoxc11', 'Hoxa11', 'Hoxa11os',
       'Hoxd11', 'Hoxc12']

Make a definition we can use to easily plot

In [ ]:
def plotfigure(merged, lower, gene, mark = ''):
    plt.figure(figsize=(10,10))
    fig = plt.scatter(x=merged['V1'], y=merged['V2'], color = 'grey')#cols[0]) #, alpha=0.2)
    
#    fig = sns.scatterplot(x=lower['V1'], y=lower['V2'], hue = merged['celltype'],
 #                         palette=sns.color_palette("tab20", 13), alpha = 0.7)
    fig = plt.scatter(x=lower['V1'], y=lower['V2'], color = cols[0], alpha = 0.7) 
    
    fig = plt.scatter(x=merged[merged[gene] > 0]['V1'], 
                      y=merged[merged[gene] > 0]['V2'],color = cols[2])
    
    fig = plt.scatter(x=merged[merged[gene] > 1]['V1'], 
                      y=merged[merged[gene] > 1]['V2'], color = cols[3])

    fig = plt.scatter(x=merged[merged[gene] > 2]['V1'], 
                      y=merged[merged[gene] > 2]['V2'], color = cols[4])

    fig = plt.scatter(x=merged[merged[gene] > 3]['V1'], 
                      y=merged[merged[gene] > 3]['V2'], color = cols[5])

    fig = plt.scatter(x=merged[merged[gene] > 4]['V1'], 
                      y=merged[merged[gene] > 4]['V2'], color = cols[6])

    fig = plt.scatter(x=merged[merged[gene] > 5]['V1'], 
                      y=merged[merged[gene] > 5]['V2'], color = cols[7])

    fig = plt.scatter(x=merged[merged[gene] > 6]['V1'], 
                      y=merged[merged[gene] > 6]['V2'], color = cols[8])
    plt.title(gene)
    #to save the figure:
    fig.figure.savefig(pathToData + 'plots/' + mark + gene + '.png', dpi=300)

    

Loop over gene lists, plot everything and save the figures.

In [ ]:
for gene in markergenes: 
    plotfigure(merged_TSS_K4, lower_TSS_K4, gene, mark = 'k4me3_')

In [ ]:
for gene in hox: 
    plotfigure(merged_TSS_K4, lower_TSS_K4, gene, mark = 'k4me3_')

K27 TSS plots:

In [ ]:
for gene in markergenes: 
    plotfigure(merged_TSS_K27, lower_TSS_K27, gene, mark = 'k27me3_TSS_')

In [ ]:
for gene in hox: 
    plotfigure(merged_TSS_K27, lower_TSS_K27, gene, mark = 'k27me3_TSS_')